In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt     
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
# load dataset
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
#from sklearn.cross_validation import  train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
#from sklearn.grid_search import GridSearchCV2 #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#loading train and test data-set

train = pd.read_csv("/content/drive/My Drive/IDAO 2020/IDAO 2020/train.csv",header=0)
test = pd.read_csv("/content/drive/My Drive/IDAO 2020/IDAO 2020/Track 1/test.csv",header=0)
print(train.shape,test.shape)

(649912, 15) (284071, 9)


In [4]:


def date_time_extract(a,count):
    if count == 0:
        print('extracting date, time, hour......for train data')
        train['hour'] = pd.DatetimeIndex(a).hour
        train['minute'] = pd.DatetimeIndex(a).minute
        train['second'] = pd.DatetimeIndex(a).second
        train['day'] = pd.DatetimeIndex(a).day
        print('extraction of date_time_hour done.........!!')
    else:
        print('extracting date, time, hour......for test data')
        test['hour'] = pd.DatetimeIndex(a).hour
        test['minute'] = pd.DatetimeIndex(a).minute
        test['second'] = pd.DatetimeIndex(a).second
        test['day'] = pd.DatetimeIndex(a).day
        print('extraction of date_time_hour done.........!!')
    
    count+=1
    
    
    

date_time_extract(train['epoch'],0)
date_time_extract(test['epoch'],1)

extracting date, time, hour......for train data
extraction of date_time_hour done.........!!
extracting date, time, hour......for test data
extraction of date_time_hour done.........!!


In [0]:
def get_acceleration(a,b,c,count):
    a1 = []
    a2 = []
    a3 = []
    
    #a1.append(0)
    #a2.append(0)
    #a3.append(0)
    
    a1 = np.asarray(a)
    a2 = np.asarray(b)
    a3 = np.asarray(c)
    
    b1 = []
    b2 = []
    b3 = []
    
    if count == 0:
    
        for i in range(len(np.asarray(train['hour']))-1):
            #print('extracting acceleration in 3-D....')
            b1.append((a1[i+1]-a1[i]))
            b2.append((a2[i+1]-a2[i]))
            b3.append((a3[i+1]-a3[i]))
        
        b1.append(a[len(a)-1])
        b2.append(b[len(a)-1])
        b3.append(c[len(a)-1])
    
        print('extraction done!!')
    
        print('loading acceleration values in train...')
        train['a_x'] = np.asarray(b1)
        train['a_y'] = np.asarray(b2)
        train['a_z'] = np.asarray(b3)
        print('loaded acc values in train...')
    
    else:
        
        for i in range(len(np.asarray(test['hour']))-1):
            #print('extracting acceleration in 3-D....')
            b1.append((a1[i+1]-a1[i]))
            b2.append((a2[i+1]-a2[i]))
            b3.append((a3[i+1]-a3[i]))
        
        b1.append(a[len(a)-1])
        b2.append(b[len(a)-1])
        b3.append(c[len(a)-1])
        
        
        
        
        print('loading acceleration values in test')
        test['a_x'] = np.asarray(b1)
        test['a_y'] = np.asarray(b2)
        test['a_z'] = np.asarray(b3)
        print('loaded acc values in test')
    
    
        

In [6]:
get_acceleration(train['Vx_sim'],train['Vy_sim'],train['Vz_sim'],0)
get_acceleration(test['Vx_sim'],test['Vy_sim'],test['Vz_sim'],1)

extraction done!!
loading acceleration values in train...
loaded acc values in train...
loading acceleration values in test
loaded acc values in test


In [0]:
def get_features(t,count):
    if count == 0:
        feature_df_train = train[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]
        return feature_df_train
    
    else:
        feature_df_test = test[['Vx_sim', 'Vy_sim', 'Vz_sim', 'x_sim', 'y_sim', 'z_sim','a_x','a_y','a_z']]
        return feature_df_test
    

In [0]:
x_train = get_features(train,0)
x_train = np.asarray(x_train)
x_test = get_features(test,0)
x_test = np.asarray(x_test)

In [0]:
def get_target_variable(x):
    return np.asarray(x)


In [0]:
y_x = get_target_variable(train['x'])
y_y = get_target_variable(train['y'])
y_z = get_target_variable(train['z'])

y_vx = get_target_variable(train['Vx'])
y_vy = get_target_variable(train['Vy'])
y_vz = get_target_variable(train['Vz'])

In [0]:
x_train_split,x_val_split,y_train_x,y_val_x,y_train_y,y_val_y,y_train_z,y_val_z,y_train_vx, y_val_vx,y_train_vy,y_val_vy, y_train_vz,y_val_vz = train_test_split(x_train,y_x,y_y,y_z,y_vx,y_vy,y_vz,test_size = 0.3,random_state = 5)

In [12]:
#using neural network
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor


NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))




# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Using TensorFlow backend.






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1280      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 512)          

In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [14]:
NN_model.fit(x_train, y_x, epochs=30, batch_size=128, validation_split = 0.2, callbacks=callbacks_list)




Train on 519929 samples, validate on 129983 samples
Epoch 1/30





519929/519929 [==============================] - 165s 318us/step - loss: 3221.0762 - mean_absolute_error: 3221.0762 - val_loss: 3917.0329 - val_mean_absolute_error: 3917.0329

Epoch 00001: val_loss improved from inf to 3917.03286, saving model to Weights-001--3917.03286.hdf5
Epoch 2/30
519929/519929 [==============================] - 165s 317us/step - loss: 3073.5729 - mean_absolute_error: 3073.5729 - val_loss: 3868.1020 - val_mean_absolute_error: 3868.1020

Epoch 00002: val_loss improved from 3917.03286 to 3868.10204, saving model to Weights-002--3868.10204.hdf5
Epoch 3/30
519929/519929 [==============================] - 164s 316us/step - loss: 3037.4541 - mean_absolute_error: 3037.4541 - val_loss: 3847.7594 - val_mean_absolute_error: 3847.7594

Epoch 00003: val_loss improved from 3868.10204 to 3847.75939, saving model to Weights-003--3847.75939.hdf5
Epoch 4/30
519929/519929 [==============================] - 164s 

KeyboardInterrupt: ignored

In [0]:
##### defining the score

def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    beta = np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))
    return 100*(1-beta)



In [0]:

"""

# this is best till now
#n_estimators = 2000

def xgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x):
    #n_Estimators = 10000
    model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.1,
                 max_depth=6,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
    #print('all parameters enabled--->')
    print('train for y_train_x !!------>')
    
    model.fit(x_train_split,y_train_x)
    
    print('training for y_train_x completed-------->prediction started for y_val_x')
    
    a = model.predict(x_val_split)
    
    print('prediction for y_val_x completed------> evaluating result for validation data')
    
    print('the score for validation obtained is:',smape(a,y_val_x))
    
    print('evaluating results for train---->')
    a = model.predict(x_train_split)
    print('the score for train obtained is:',smape(a,y_train_x))
    
    
    



In [0]:
# xgb model for y_train_x
xgb_model_evaluation(x_train_split,x_val_split,y_train_x,y_val_x)

In [0]:
# xgb model for y_train_y
xgb_model_evaluation(x_train_split,x_val_split,y_train_y,y_val_y)

In [0]:
# xgb model for y_train_x
xgb_model_evaluation(x_train_split,x_val_split,y_train_z,y_val_z)

In [0]:
# xgb model for y_train_vx
xgb_model_evaluation(x_train_split,x_val_split,y_train_vx,y_val_vx)

In [0]:
# xgb model for y_train_vy
xgb_model_evaluation(x_train_split,x_val_split,y_train_vy,y_val_vy)

In [0]:
# xgb model for y_train_vz
xgb_model_evaluation(x_train_split,x_val_split,y_train_vz,y_val_vz)